In [1]:
from pathlib import Path
import pandas as pd

RAW_DIR = Path('/Users/powerbai/Desktop/Power Bai/Data Analyst Bootcamp/Portfolio Projects/bike-share-demand-analysis/data_raw')

csv_files = sorted(RAW_DIR.glob("*.csv"))
print(f"Found {len(csv_files)} csv files")

usecols = [
    "ride_id",
    "started_at",
    "ended_at",
    "rideable_type",
    "member_casual"
]

dfs = []
for fp in csv_files:
    df = pd.read_csv(fp, usecols = lambda c: c in usecols)
    dfs.append(df)

trips = pd.concat(dfs, ignore_index = True)
print(trips.shape)
trips.head()

Found 18 csv files
(9008599, 5)


,ride_id,rideable_type,started_at,ended_at,member_casual
0,2658E319B13141F9,electric_bike,2024-07-11 08:15:14.784,2024-07-11 08:17:56.335,casual
1,B2176315168A47CE,electric_bike,2024-07-11 15:45:07.851,2024-07-11 16:06:04.243,casual
2,C2A9D33DF7EBB422,electric_bike,2024-07-11 08:24:48.192,2024-07-11 08:28:05.237,casual
3,8BFEA406DF01D8AD,electric_bike,2024-07-11 08:46:06.864,2024-07-11 09:14:11.664,casual
4,ECD3EF02E5EB73B6,electric_bike,2024-07-11 18:18:16.588,2024-07-11 18:30:20.288,casual


In [2]:
trips["started_at"] = pd.to_datetime(trips["started_at"], errors = "coerce")
trips["ended_at"] = pd.to_datetime(trips["ended_at"], errors = "coerce")

In [3]:
trips["started_at"].isna().mean(), trips["ended_at"].isna().mean()

(np.float64(0.0), np.float64(0.0))

In [4]:
trips = trips.dropna(subset=["started_at", "ended_at"])
trips = trips[trips["ended_at"] >= trips["started_at"]]

In [5]:
trips["ride_length_min"] = (
    (trips["ended_at"] - trips["started_at"])
    .dt.total_seconds() / 60
)

trips["ride_length_min"].describe()

count    9.008527e+06
mean     1.639836e+01
std      5.638444e+01
min      7.000000e-04
25%      5.462850e+00
50%      9.538267e+00
75%      1.679515e+01
max      1.574900e+03
Name: ride_length_min, dtype: float64

In [6]:
daily = (
    trips
    .assign(date=trips["started_at"].dt.floor("D"))
    .groupby("date", as_index = False)
    .agg(
        daily_ride_count = ("ride_id", "count"),
        avg_ride_length_min = ("ride_length_min", "mean")
    )
    .sort_values("date")
)

daily.head(), daily.tail(), daily["daily_ride_count"].describe(), daily["date"].min(), daily["date"].max(), daily.shape

(        date  daily_ride_count  avg_ride_length_min
 0 2024-06-29                 3          1499.766583
 1 2024-06-30               234           483.388354
 2 2024-07-01             21555            18.747473
 3 2024-07-02             23031            17.405056
 4 2024-07-03             25046            18.489790,
           date  daily_ride_count  avg_ride_length_min
 546 2025-12-27              4197            16.762762
 547 2025-12-28              1997            14.200074
 548 2025-12-29              2388            12.260312
 549 2025-12-30              3071            14.569616
 550 2025-12-31              2952            10.581505,
 count      551.000000
 mean     16349.413793
 std       8786.593607
 min          3.000000
 25%       7970.000000
 50%      17124.000000
 75%      23928.500000
 max      34698.000000
 Name: daily_ride_count, dtype: float64,
 Timestamp('2024-06-29 00:00:00'),
 Timestamp('2025-12-31 00:00:00'),
 (551, 3))

In [7]:
OUT_PATH = Path('/Users/powerbai/Desktop/Power Bai/Data Analyst Bootcamp/Portfolio Projects/bike-share-demand-analysis/data_processed/daily_demand.csv')
OUT_PATH.parent.mkdir(parents = True, exist_ok = True)
daily.to_csv(OUT_PATH, index = False)
OUT_PATH

PosixPath('/Users/powerbai/Desktop/Power Bai/Data Analyst Bootcamp/Portfolio Projects/bike-share-demand-analysis/data_processed/daily_demand.csv')

In [8]:
import pandas as pd

check = pd.read_csv(
    '/Users/powerbai/Desktop/Power Bai/Data Analyst Bootcamp/Portfolio Projects/bike-share-demand-analysis/data_processed/daily_demand.csv',
    parse_dates = ["date"]
)

check.head(), check.shape

(        date  daily_ride_count  avg_ride_length_min
 0 2024-06-29                 3          1499.766583
 1 2024-06-30               234           483.388354
 2 2024-07-01             21555            18.747473
 3 2024-07-02             23031            17.405056
 4 2024-07-03             25046            18.489790,
 (551, 3))